In [9]:
import plotly.offline as plt
from plotly import tools
import plotly.graph_objs as go
from plotly.graph_objs import Layout
from plotly.graph_objs.graph_objs import Bar
from ipywidgets import *
import ipywidgets as widgets

plt.init_notebook_mode(connected=True)


In [10]:

class MultiHist(object):
    """Class for showing stacked or single histograms using plotly"""
    def __init__(self, titration):
        """Define attributs.
        argument 'titration' is a two dimensions list (should be a titration object from shiftome after developpement)"""
        self.titration=titration
        self.x=self.titration[0]#list of abscissa (dimension equal to the number of plots)
        self.y=self.titration[1]#ordinates (dimension equal to the number of plots)
        self.slider=widgets.FloatSlider(max=5, step=0.001, orientation="horizontal") #widget modifying cutoff as a slider
        self.textfloat=BoundedFloatText(max=5, min=0, step=0.001) # widget modifying cutoff as a text box
        self.link=widgets.jslink((self.textfloat, 'value'), (self.slider, 'value')) # link between slider and text box
        self.stepSelector=widgets.Dropdown(options=['All']+[index+1 for index,step in enumerate(self.titration[0])],
                                           value='All',description='Step:',disabled=False,) # widget for selecting 'All' or one step

        
    def updateAttr(self, stepInt):
        """Changes attributs x and y coordinates.
        argument 'stepInt' is either a string or an integer representing 'All' one step"""
        if stepInt == 'All':
            self.x=self.titration[0]
            self.y=self.titration[1]
        else:
            self.x=[self.titration[0][stepInt-1]]
            self.y=[self.titration[1][stepInt-1]]
       
        
    def updateHist(self,stepInt, slider, textbox, ):
        """Method for tracing stacked or single histogram in one or more subplots.
        Creates one or more histogram with personalized parameters such as color bars and annotations regarding cutoff value.
        Defines a layout in which a 
        cutoff line is traced and a global legend and y scale for all subplots.
        argument 'stepInt' is either a string or an integer representing 'All or one step.
        argument 'slider' and 'textbox' are float numbers representing the same cutoff value."""
        self.updateAttr(stepInt=stepInt) #updates attributs
        fig = tools.make_subplots(rows=len(self.y), cols=1, print_grid=False) # one-column stacked histograms
        ##global plot parameters
        fig['layout'].update(title='Titration step 1 to '+str(len(self.y)),shapes=[ 
                                    {
                                        'type': 'line',
                                        'x0':-1, #x position of the cutoff line
                                        'y0': slider,
                                        'x1':len(self.y[0]),
                                        'y1': slider,
                                        'xref':"x", #plot number
                                        'yref':"y",
                                        'line': {
                                            'color': 'rgb(255,0,0)',
                                            'width': 1,
                                            'dash':'dashdot',
                                        },
                                    },
                                ]*len(self.y),showlegend=False,
                                    annotations=[{
                                            'x':-1, #x position of the annotation
                                            'y':2,
                                            'xref':'x', #plot number
                                            'yref':'y',
                                            'text':'max=5',
                                            'showarrow':False,
                                            'font':{
                                                'family':'Courier New, monospace',
                                                'size':11,
                                                'color':'#ffffff'
                                            },
                                            
                                            'bordercolor':'rgb(255,255,255)',
                                            'borderwidth':2,
                                            'borderpad':4,
                                            'bgcolor':'rgb(0,0,0)',
                                            'opacity':0.7
                                    }
                                    ]*len(self.y)+[{
                                            'xref': 'paper', #plot number
                                            'yref': 'paper',
                                            'x': 0, #x poition of x ax legend
                                            'xanchor': 'right',
                                            'y': 1,
                                            'yanchor': 'bottom',
                                            'text': 'Chemical shift intensity',
                                            'showarrow': False,
                                            'font':{'size':16,},
                                          }, {
                                            'xref': 'paper',
                                            'yref': 'paper',
                                            'x': 1,
                                            'xanchor': 'left',
                                            'y': 0,
                                            'yanchor': 'top',
                                            'text': 'Amino acid number',
                                            'showarrow': False,
                                            'font':{'size':16,}
                                          }],
                                    width=1000, #width of plotly interface
                                    height=1000,
                                    margin=go.Margin(
                                        l=200,
                                        r=200,
                                        b=100,
                                        t=100,
                                        pad=4
    )
                )
        ##
        ##personalized plot parameters
        for stepIndex,step in enumerate(self.y):
            color=[] #color list for all bars
            for index,elm in enumerate(step):
                if elm > slider:
                    color.append('rgb(255,128,0)')
                else:
                    color.append('rgb(0,128,255)')
            trace1 = go.Bar(
                    x=self.x[stepIndex],
                    y=self.y[stepIndex],
                    marker=dict(
                        color=color),
                    name='Step '+str(stepIndex+1),
                )
            fig.append_trace(trace1, stepIndex+1, 1)
            fig['layout']['shapes'][stepIndex]['xref']="x"+str(stepIndex+1) #cutoff line affectation
            fig['layout']['shapes'][stepIndex]['yref']="y"+str(stepIndex+1)
            fig['layout']['annotations'][stepIndex]['xref']="x"+str(stepIndex+1) #annotation affectation
            fig['layout']['annotations'][stepIndex]['yref']="y"+str(stepIndex+1)
            fig['layout']['annotations'][stepIndex]['x']=(len(self.x[stepIndex]))/2 #annotation :middle plot position
            fig['layout']['annotations'][stepIndex]['y']=1.5*max(self.y[-1])#annotation :top plot position
            fig['layout']['annotations'][stepIndex]['text']="Step " +str(stepIndex+1)
            fig['layout']['yaxis'+str(stepIndex+1)].update(range=[0,1.8*max(self.y[-1])])#harmonized y scale for all subplots
        ##one step histogram specicifc parmameters
        if stepInt != 'All': 
            fig['layout'].update(xaxis={'showline':True, 'showgrid':True,'zeroline':True}, 
                                 yaxis={'range':[0,max(self.y[stepIndex])],'showline':True,'zeroline':False}) #show x and y axes
            fig['layout']['annotations']=[] #delete annotation
            fig['layout']['title']="Titration step "+str(stepInt)
        ##
        ##
        plt.iplot(fig)



In [11]:
titrationExample=[[[chr(carac) for carac in range(1,100)]]*9,
[[0.01*num for num in range(1,100)],
  [0.02*num for num in range(1,100)], 
  [0.03*num for num in range(1,100)], 
  [0.04*num for num in range(1,100)],
  [0.05*num for num in range(1,100)], 
  [0.06*num for num in range(1,100)],
  [0.07*num for num in range(1,100)], 
  [0.08*num for num in range(1,100)],
  [0.09*num for num in range(1,100)]]]
h=MultiHist(titrationExample)
widgets.interactive(h.updateHist, stepInt=h.stepSelector, slider=h.slider, textbox=h.textfloat, )

A Jupyter Widget